In [36]:
using LinearAlgebra
using Distributions
using Optim
using Random
using StatsFuns
using JuMP
using MosekTools
using JLD2

include("ETO.jl")
include("RO.jl")
include("Data.jl")
include("Estimate.jl")
include("Performance.jl")
include("Plot_Figures.jl")

plot_boxplot (generic function with 1 method)

In [2]:
current_dir = pwd()
parent_dir = dirname(current_dir)
grand_pa_dir = dirname(parent_dir)
data_dir = string(grand_pa_dir, "/Data/")
if !isdir(data_dir)
    mkpath(data_dir)
end

## Functions

In [3]:
function generate_Input_Data(S_train,S_test,iterations, N, N_u, K, offdiag_sign,max_offdiag,P_bar)
    Input_Data = Dict()
    for iter in 1:iterations
        A_true, B_true = Generate_Coef(N_u, N,max_offdiag,offdiag_sign);
        U_train, P_train = Generate_Feat_Data(N_u, N, S_train);
        U_test, P_test = Generate_Feat_Data(N_u, N, S_test);

        Input_Data["iter=$(iter)_Obs_Feat"] = U_test[1,:];
        Input_Data["iter=$(iter)_A_true"] = A_true;
        Input_Data["iter=$(iter)_B_true"] = B_true;
        Input_Data["iter=$(iter)_P_dag"] = round.(rand(N, K) .* P_bar; digits=2);
        Input_Data["iter=$(iter)_U_train"] = U_train;
        Input_Data["iter=$(iter)_P_train"] = P_train;

        A_hat, B_hat = Estimate_MNL_Para(U_train, P_train, S_train, N, N_u, N, A_true, B_true);

        Input_Data["iter=$(iter)_A_hat"] = A_hat
        Input_Data["iter=$(iter)_B_hat"] = B_hat
    end
    return Input_Data
end

generate_Input_Data (generic function with 1 method)

In [4]:
function Run_Oracle(iterations, N, N_u, K, Input_Data,print_flag)
    RST_ = Dict()
    for iter in 1:iterations
        Obs_Feat = Input_Data["iter=$(iter)_Obs_Feat"]
        A_true = Input_Data["iter=$(iter)_A_true"]
        B_true = Input_Data["iter=$(iter)_B_true"]
        P_dag = Input_Data["iter=$(iter)_P_dag"]
        
        obj_ETO,X_ETO,time_ETO = Solve_ETO(N,N_u,K,A_true,B_true,Obs_Feat,P_dag)
        rev_ETO, price_ETO = compute_oof(X_ETO, A_true, B_true, Obs_Feat, P_dag)
        if print_flag
            println("iter=$(iter): rev_Oracle = ",round(rev_ETO,digits=6),", price_Oracle = ",price_ETO)
        end
        RST_["iter=$(iter)_Time"] = time_ETO
        RST_["iter=$(iter)_Rev"] = rev_ETO
        RST_["iter=$(iter)_Price"] = price_ETO
        RST_["iter=$(iter)_Obj"] = obj_ETO
    end
    return RST_
end

Run_Oracle (generic function with 1 method)

In [5]:
function Run_ETO(iterations, N, N_u, K, Input_Data,print_flag)
    RST_ETO = Dict()
    for iter in 1:iterations
        Obs_Feat = Input_Data["iter=$(iter)_Obs_Feat"]
        A_hat = Input_Data["iter=$(iter)_A_hat"]
        B_hat = Input_Data["iter=$(iter)_B_hat"]
        A_true = Input_Data["iter=$(iter)_A_true"]
        B_true = Input_Data["iter=$(iter)_B_true"]
        P_dag = Input_Data["iter=$(iter)_P_dag"]
        
        obj_ETO,X_ETO,time_ETO = Solve_ETO(N,N_u,K,A_hat,B_hat,Obs_Feat,P_dag)
        rev_ETO, price_ETO = compute_oof(X_ETO, A_true, B_true, Obs_Feat, P_dag)
        if print_flag
            println("iter=$(iter): rev_ETO = ",round(rev_ETO,digits=6),", price_ETO = ",price_ETO)
        end
        RST_ETO["iter=$(iter)_Time"] = time_ETO
        RST_ETO["iter=$(iter)_Rev"] = rev_ETO
        RST_ETO["iter=$(iter)_Price"] = price_ETO
        RST_ETO["iter=$(iter)_Obj"] = obj_ETO
    end
    return RST_ETO
end

Run_ETO (generic function with 1 method)

In [6]:
function search_opt_price(N,p_lb,p_ub,b_n)
    model = Model(Mosek.Optimizer)
    set_attribute(model, "QUIET", true)
    # 定义变量
    @variable(model, price[1:N])                      # y_{nk}
    @constraint(model, price .>= p_lb)
    @constraint(model, price .<= p_ub)
    @objective(model, Max,b_n' * price )
    optimize!(model)
    obj_val = objective_value(model)
    return obj_val
end

search_opt_price (generic function with 1 method)

In [7]:
function Calculate_Hyper_Param(RO_coef_all, iterations, N, N_u, K, Input_Data)
    for iter in 1:iterations
        Obs_Feat = Input_Data["iter=$(iter)_Obs_Feat"]
        # A_true = Input_Data["iter=$(iter)_A_true"]
        # B_true = Input_Data["iter=$(iter)_B_true"]
        P_dag = Input_Data["iter=$(iter)_P_dag"]
        A_hat = Input_Data["iter=$(iter)_A_hat"]
        B_hat = Input_Data["iter=$(iter)_B_hat"]

        for RO_coef in RO_coef_all
            A_lb = A_hat .- RO_coef .* abs.(A_hat);
            A_ub = A_hat .+ RO_coef .* abs.(A_hat);
            B_lb = B_hat .- RO_coef .* abs.(B_hat);
            B_ub = B_hat .+ RO_coef .* abs.(B_hat);
            
            p_ub = vec(maximum(P_dag,dims=2))
            p_lb = vec(minimum(P_dag,dims=2))
            p_max = maximum(p_ub)
            p_min = minimum(p_lb)
            Obs_Feat_Trun = [max(-Obs_Feat[ind],0) for ind in 1:N_u]
            psi_lb = zeros(N)
            for n in 1:N
                b_n = B_lb[n,:]
                obj_n = search_opt_price(N,p_lb,p_ub,b_n)
                psi_lb[n] = max(-10000,-exp(-Obs_Feat_Trun' * (A_ub[n,:] - A_lb[n,:]) + Obs_Feat' * A_lb[n,:] + obj_n)*(p_max-p_min))
            end        

            Input_Data["iter=$(iter)_RO_coef=$(RO_coef)_psi_lb"] = psi_lb
            Input_Data["iter=$(iter)_RO_coef=$(RO_coef)_psi_ub"] = zeros(N)
            Input_Data["iter=$(iter)_RO_coef=$(RO_coef)_phi_lb"] = [p_min - p_max for i in 1:N]
            Input_Data["iter=$(iter)_RO_coef=$(RO_coef)_phi_ub"] = zeros(N)

            Input_Data["iter=$(iter)_RO_coef=$(RO_coef)_A_lb"] = A_lb
            Input_Data["iter=$(iter)_RO_coef=$(RO_coef)_A_ub"] = A_ub
            Input_Data["iter=$(iter)_RO_coef=$(RO_coef)_B_lb"] = B_lb
            Input_Data["iter=$(iter)_RO_coef=$(RO_coef)_B_ub"] = B_ub
        end
    end
    return Input_Data
end

Calculate_Hyper_Param (generic function with 1 method)

In [8]:
function Run_RO(RO_coef, iterations, N, N_u, K, Input_Data,psi_coef,model_name,print_flag)
    RST_RO = Dict()
    for iter in 1:iterations
        Obs_Feat = Input_Data["iter=$(iter)_Obs_Feat"]
        A_true = Input_Data["iter=$(iter)_A_true"]
        B_true = Input_Data["iter=$(iter)_B_true"]
        P_dag = Input_Data["iter=$(iter)_P_dag"]

        psi_lb = Input_Data["iter=$(iter)_RO_coef=$(RO_coef)_psi_lb"]
        psi_ub = Input_Data["iter=$(iter)_RO_coef=$(RO_coef)_psi_ub"]
        phi_lb = Input_Data["iter=$(iter)_RO_coef=$(RO_coef)_phi_lb"]
        phi_ub = Input_Data["iter=$(iter)_RO_coef=$(RO_coef)_phi_ub"]

        A_lb = Input_Data["iter=$(iter)_RO_coef=$(RO_coef)_A_lb"]
        A_ub = Input_Data["iter=$(iter)_RO_coef=$(RO_coef)_A_ub"]
        B_lb = Input_Data["iter=$(iter)_RO_coef=$(RO_coef)_B_lb"]
        B_ub = Input_Data["iter=$(iter)_RO_coef=$(RO_coef)_B_ub"]
        if model_name == "Two_Side"
            obj_RO,X_RO,time_RO = Solve_RO(N,N_u,K,A_lb,A_ub,B_lb,B_ub,Obs_Feat,P_dag,psi_lb,psi_ub,phi_lb,phi_ub)
        else
            obj_RO,X_RO,time_RO = Solve_RO_one_side_exp(N,N_u,K,A_lb,A_ub,B_lb,B_ub,Obs_Feat,P_dag,psi_lb,psi_ub)
        end
        
        rev_RO, price_RO = compute_oof(X_RO, A_true, B_true, Obs_Feat, P_dag)
        if print_flag
            println("iter=$(iter): rev_RO = ",round(rev_RO,digits=6),", price_RO = ",price_RO)
        end
        RST_RO["iter=$(iter)_Time"] = time_RO
        RST_RO["iter=$(iter)_Rev"] = rev_RO
        RST_RO["iter=$(iter)_Price"] = price_RO
        RST_RO["iter=$(iter)_Obj"] = obj_RO
    end
    return RST_RO
end

Run_RO (generic function with 1 method)

In [9]:
function Run_RO_with_RO_coef(RO_coef, iterations, N, N_u, K, Input_Data, psi_coef,model_name,print_flag)
    RST_RO = Run_RO(RO_coef, iterations, N, N_u, K, Input_Data,psi_coef,model_name,print_flag);
    return RST_RO
end

Run_RO_with_RO_coef (generic function with 1 method)

In [10]:
function set_Params(N, N_u, K, S_train, S_test, offdiag_sign, max_offdiag, P_bar, RO_coef_all, seed)
    Params = Dict()
    Params["N"] = N
    Params["N_u"] = N_u
    Params["K"] = K
    Params["S_train"] = S_train
    Params["S_test"] = S_test
    Params["offdiag_sign"] = offdiag_sign
    Params["max_offdiag"] = max_offdiag
    Params["P_bar"] = P_bar
    Params["RO_coef_all"] = RO_coef_all
    Params["seed"] = seed
    return Params
end

set_Params (generic function with 1 method)

## Evaluate

#### Baseline Parameters

In [11]:
project_name = "Assortment_Pricing_Simulated_Data_Test/"
seed = 1;
N = 3;
N_u = 1;
K = 10;
S_test = 1000;
P_bar = 5.0;
iterations = 100;
# RO_coef_all = [0.0,0.01,0.05,0.1,0.12,0.14,0.16,0.18,0.2];
RO_coef_all = [0.0,0.01,0.05,0.1,0.12];

### offdiag is mix

In [12]:
S_train = 100;
offdiag_sign = "mix";
max_offdiag = 1.0;
Params = set_Params(N, N_u, K, S_train, S_test, offdiag_sign, max_offdiag, P_bar, RO_coef_all, seed);

In [13]:
sub_file_name = "N=$(N)_N_u=$(N_u)_K=$(K)_S_train=$(S_train)_offdiag_sign=$(offdiag_sign)_max_offdiag=$(max_offdiag)/"
this_data_file = string(data_dir,project_name,sub_file_name)
if !isdir(this_data_file)
    mkpath(this_data_file)
end
save(string(this_data_file, "Params.jld2"), Params);

- Generate Data

In [14]:
Random.seed!(seed)
Input_Data = generate_Input_Data(S_train,S_test,iterations, N, N_u, K, offdiag_sign,max_offdiag,P_bar);
Input_Data = Calculate_Hyper_Param(RO_coef_all, iterations, N, N_u, K, Input_Data);
save(string(this_data_file, "Input_Data.jld2"), Input_Data);

In [15]:
Input_Data = Calculate_Hyper_Param(RO_coef_all, iterations, N, N_u, K, Input_Data);

- Run Oracle

In [16]:
print_flag=true
RST_Oracle = Run_Oracle(iterations, N, N_u, K, Input_Data,print_flag);
Rev_Oracle = [RST_Oracle["iter=$(iter)_Rev"] for iter in 1:iterations];
save(string(this_data_file, "RST_Oracle.jld2"), RST_Oracle);

iter=1: rev_Oracle = 0.613145, price_Oracle = [0.93, 3.63, 0.5]
iter=2: rev_Oracle = 1.410996, price_Oracle = [4.82, 2.68, 4.14]
iter=3: rev_Oracle = 1.919721, price_Oracle = [4.96, 2.24, 3.08]
iter=4: rev_Oracle = 1.708866, price_Oracle = [4.42, 4.7, 2.57]
iter=5: rev_Oracle = 0.929387, price_Oracle = [4.8, 1.45, 0.57]
iter=6: rev_Oracle = 0.564078, price_Oracle = [2.1, 0.31, 4.51]
iter=7: rev_Oracle = 1.199992, price_Oracle = [0.79, 4.62, 2.58]
iter=8: rev_Oracle = 1.480268, price_Oracle = [2.0, 4.81, 3.92]
iter=9: rev_Oracle = 0.567877, price_Oracle = [4.2, 0.6, 2.14]
iter=10: rev_Oracle = 0.959138, price_Oracle = [1.0, 1.55, 4.68]
iter=11: rev_Oracle = 2.024549, price_Oracle = [4.96, 4.79, 2.22]
iter=12: rev_Oracle = 1.624291, price_Oracle = [4.82, 1.89, 3.21]
iter=13: rev_Oracle = 1.581446, price_Oracle = [4.51, 2.16, 4.94]
iter=14: rev_Oracle = 1.203803, price_Oracle = [3.25, 4.89, 0.42]
iter=15: rev_Oracle = 0.759632, price_Oracle = [4.34, 0.82, 1.19]
iter=16: rev_Oracle = 1.962

- Run ETO

In [17]:
RST_ETO = Run_ETO(iterations, N, N_u, K, Input_Data,print_flag);
Rev_ETO = [RST_ETO["iter=$(iter)_Rev"] for iter in 1:iterations]
println("Rev_ETO/Rev_Oracle = ",round.(Rev_ETO./Rev_Oracle,digits=4))
save(string(this_data_file, "RST_ETO.jld2"), RST_ETO);

iter=1: rev_ETO = 0.13871, price_ETO = [0.93, 2.81, 3.77]
iter=2: rev_ETO = 0.301583, price_ETO = [4.82, 4.74, 0.55]
iter=3: rev_ETO = 1.093242, price_ETO = [3.85, 0.25, 5.0]
iter=4: rev_ETO = 0.033693, price_ETO = [1.0, 4.7, 4.86]
iter=5: rev_ETO = 0.500114, price_ETO = [4.8, 2.9, 0.31]
iter=6: rev_ETO = 0.488278, price_ETO = [0.73, 0.31, 4.51]
iter=7: rev_ETO = 0.143536, price_ETO = [0.79, 4.62, 4.94]
iter=8: rev_ETO = 1.390151, price_ETO = [2.79, 4.81, 3.92]
iter=9: rev_ETO = 0.079531, price_ETO = [0.33, 0.6, 4.86]
iter=10: rev_ETO = 0.522185, price_ETO = [1.82, 2.1, 4.68]
iter=11: rev_ETO = 1.668173, price_ETO = [4.96, 4.79, 1.71]
iter=12: rev_ETO = 0.751155, price_ETO = [4.82, 0.02, 4.32]
iter=13: rev_ETO = 1.148249, price_ETO = [4.51, 0.8, 4.94]
iter=14: rev_ETO = 0.542763, price_ETO = [3.98, 4.89, 4.9]
iter=15: rev_ETO = 0.028785, price_ETO = [4.34, 0.11, 4.88]
iter=16: rev_ETO = 1.118574, price_ETO = [1.12, 5.0, 4.84]
iter=17: rev_ETO = 0.297334, price_ETO = [4.93, 4.9, 0.6]
it

- Run RO under different uncertainty set

In [18]:
model_name="Two_Side"
RST_RO = Dict(); Rev_RO = Dict();
for RO_coef in RO_coef_all
    println("Running RO with RO_coef = ",RO_coef)
    RST_RO_this = Run_RO_with_RO_coef(RO_coef, iterations, N, N_u, K, Input_Data, 0.0,model_name,false);
    RST_RO["RO_coef=$(RO_coef)"] = RST_RO_this
    Rev_RO["RO_coef=$(RO_coef)"] = [RST_RO_this["iter=$(iter)_Rev"] for iter in 1:iterations]
end
save(string(this_data_file, "RST_RO.jld2"), RST_RO);

Running RO with RO_coef = 0.0
Running RO with RO_coef = 0.01
Running RO with RO_coef = 0.05
Running RO with RO_coef = 0.1
Running RO with RO_coef = 0.12


In [19]:
Rev_Oracle = [RST_Oracle["iter=$(iter)_Rev"] for iter in 1:iterations];
Rev_ETO = [RST_ETO["iter=$(iter)_Rev"] for iter in 1:iterations];
Rev_RO = Dict();
for RO_coef in RO_coef_all
    RST_RO_this = RST_RO["RO_coef=$(RO_coef)"]
    Rev_RO["RO_coef=$(RO_coef)"] = [RST_RO_this["iter=$(iter)_Rev"] for iter in 1:iterations];
end

println("Mean Rev_Oracle: ", mean(Rev_Oracle));
println("Mean Rev_ETO: ", mean(Rev_ETO));
Rev_RO_Avg = [mean(Rev_RO["RO_coef=$(RO_coef)"]) for RO_coef in RO_coef_all];
println("Mean Rev_RO: ", Rev_RO_Avg);

Mean Rev_Oracle: 1.4847002193577055
Mean Rev_ETO: 0.8231057792632598
Mean Rev_RO: [0.8231057792632598, 0.8288821614618992, 0.8196493446633434, 0.845140786848969, 0.8582873676191924]


In [22]:
for RO_coef in RO_coef_all
    Rev_RO_this = Rev_RO["RO_coef=$(RO_coef)"]
    println("Average = ",round(mean(Rev_RO_this./Rev_ETO),digits=4),", Rev_RO/Rev_ETO for RO_coef=$(RO_coef) = ",round.(Rev_RO_this./Rev_ETO,digits=4),)
end

Average = 1.0, Rev_RO/Rev_ETO for RO_coef=0.0 = [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]
Average = 1.0233, Rev_RO/Rev_ETO for RO_coef=0.01 = [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.6028, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0666, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.217, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0086, 1.0, 1.0, 1.

### offdiag is positive

In [23]:
# offdiag_sign = "positive";
# Params = set_Params(N, N_u, K, S_train, S_test, offdiag_sign, max_offdiag, P_bar, RO_coef_all, seed)

In [24]:
# sub_file_name = "N=$(N)_N_u=$(N_u)_K=$(K)_S_train=$(S_train)_offdiag_sign=$(offdiag_sign)_max_offdiag=$(max_offdiag)/"
# this_data_file = string(data_dir,project_name,sub_file_name)
# if !isdir(this_data_file)
#     mkpath(this_data_file)
# end
# save(string(this_data_file, "Params.jld2"), Params);

In [25]:
# Random.seed!(seed)
# Input_Data = generate_Input_Data(S_train,S_test,iterations, N, N_u, K, offdiag_sign,max_offdiag,P_bar);
# Input_Data = Calculate_Hyper_Param(RO_coef_all, iterations, N, N_u, K, Input_Data);
# save(string(this_data_file, "Input_Data.jld2"), Input_Data);

In [26]:
# print_flag=true
# RST_Oracle = Run_Oracle(iterations, N, N_u, K, Input_Data,print_flag);
# Rev_Oracle = [RST_Oracle["iter=$(iter)_Rev"] for iter in 1:iterations];
# save(string(this_data_file, "RST_Oracle.jld2"), RST_Oracle);

In [27]:
# RST_ETO = Run_ETO(iterations, N, N_u, K, Input_Data,print_flag);
# Rev_ETO = [RST_ETO["iter=$(iter)_Rev"] for iter in 1:iterations]
# println("Rev_ETO/Rev_Oracle = ",round.(Rev_ETO./Rev_Oracle,digits=4))
# save(string(this_data_file, "RST_ETO.jld2"), RST_ETO);

In [28]:
# model_name="Two_Side"
# RST_RO = Dict(); Rev_RO = Dict();
# for RO_coef in RO_coef_all
#     println("Running RO with RO_coef = ",RO_coef)
#     RST_RO_this = Run_RO_with_RO_coef(RO_coef, iterations, N, N_u, K, Input_Data, 0.0,model_name,false);
#     RST_RO["RO_coef=$(RO_coef)"] = RST_RO_this
#     Rev_RO["RO_coef=$(RO_coef)"] = [RST_RO_this["iter=$(iter)_Rev"] for iter in 1:iterations]
# end
# save(string(this_data_file, "RST_RO.jld2"), RST_RO);

In [29]:
# for RO_coef in RO_coef_all
#     Rev_RO_this = Rev_RO["RO_coef=$(RO_coef)"]
#     println("Average = ",round(mean(Rev_RO_this./Rev_ETO),digits=4),", Rev_RO/Rev_ETO for RO_coef=$(RO_coef) = ",round.(Rev_RO_this./Rev_ETO,digits=4),)
# end

### offdiag is negative

In [30]:
# offdiag_sign = "negative";
# Params = set_Params(N, N_u, K, S_train, S_test, offdiag_sign, max_offdiag, P_bar, RO_coef_all, seed)

In [31]:
# sub_file_name = "N=$(N)_N_u=$(N_u)_K=$(K)_S_train=$(S_train)_offdiag_sign=$(offdiag_sign)_max_offdiag=$(max_offdiag)/"
# this_data_file = string(data_dir,project_name,sub_file_name)
# if !isdir(this_data_file)
#     mkpath(this_data_file)
# end
# save(string(this_data_file, "Params.jld2"), Params);

In [32]:
# print_flag=true
# RST_Oracle = Run_Oracle(iterations, N, N_u, K, Input_Data,print_flag);
# Rev_Oracle = [RST_Oracle["iter=$(iter)_Rev"] for iter in 1:iterations];
# save(string(this_data_file, "RST_Oracle.jld2"), RST_Oracle);

In [33]:
# RST_ETO = Run_ETO(iterations, N, N_u, K, Input_Data,print_flag);
# Rev_ETO = [RST_ETO["iter=$(iter)_Rev"] for iter in 1:iterations]
# println("Rev_ETO/Rev_Oracle = ",round.(Rev_ETO./Rev_Oracle,digits=4))
# save(string(this_data_file, "RST_ETO.jld2"), RST_ETO);

In [34]:
# model_name="Two_Side"
# RST_RO = Dict(); Rev_RO = Dict();
# for RO_coef in RO_coef_all
#     println("Running RO with RO_coef = ",RO_coef)
#     RST_RO_this = Run_RO_with_RO_coef(RO_coef, iterations, N, N_u, K, Input_Data, 0.0,model_name,false);
#     RST_RO["RO_coef=$(RO_coef)"] = RST_RO_this
#     Rev_RO["RO_coef=$(RO_coef)"] = [RST_RO_this["iter=$(iter)_Rev"] for iter in 1:iterations]
# end
# save(string(this_data_file, "RST_RO.jld2"), RST_RO);